#   MTCNN Face Detection - Experimento

*   Este componente utiliza a biblioteca [facenet-pytorch](https://github.com/timesler/facenet-pytorch), a qual disponibiliza o algorítimo [MTCNN](https://arxiv.org/abs/1604.02878). 

* O MTCNN possui a performance estado da arte nos benchmarks [FDDB](http://vis-www.cs.umass.edu/fddb/) e [WIDER FACE](http://shuoyang1213.me/WIDERFACE/)

*   Melhores explicações são encontradas neste [artigo do kaggle](https://www.kaggle.com/timesler/guide-to-mtcnn-in-facenet-pytorch)




## Declaração de parâmetros e hiperparâmetros

Declare parâmetros com o botão  na barra de ferramentas.<br>
A variável `dataset` possui o caminho para leitura do arquivos importados na tarefa de "Upload de dados".<br>
Você também pode importar arquivos com o botão  na barra de ferramentas.

In [ ]:
#General params 
dataset =  "/tmp/data/dataset.zip" #@param {type:"string"}

#Model Params
image_size = 160 #@param {type:"integer",label:"Tamanho da imagem",description:"Tamanho da imagem de saída em pixels. Imagem será quadrada"}
margin = 20 #@param {type:"integer",label:"Margem adicionada ao bbox",description:"Margem adicionada em relação ao número de pixels da imagem final"}
min_face_size = 20 #@param {type:"integer",label:"Menor tamanho de rosto",description:"Menor tamanho de rosto que o algorítimo irá procurar em pixels"}
factor = 0.709 #@param {type:"number",label:"Fator de escalabilidade",description:"Fator de escalabilidade para pirâmide de tamanhos de rosto"}
keep_all = True #@param [True,False] {type:"boolean",label:"Manter todas as faces",description:"Se True retorna todas as faces, se não retorna apenas a com maior probabilidade caso encontre"}
device = "cuda" #@param ["cuda","cpu"] {type:"string",label:"Ambiente",description:"Escolher entre CPU e GPU. Se escolher GPU e não houver irá substituir automaticamente por cpu"}
seed = 7 #@param {type:"integer",label:"Semente Aleatória",description:"Semente para replicabilidade dos resultados"}

# Inference_params Params
inference_batch_size = 2 #@param {type:"integer",label:"Batch size para inferência",description:"Inferência em batch para acelerar o processo."}
input_square_transformation_size = 512 #@param {type:"integer",label:"Dimensão para redimensionamento",description:"Dimensão em que todas as imagens serão redimensionadas para poderem ser procesadas em batch. Ficarão todas quadradas"}

## Acesso ao conjunto de dados

O conjunto de dados utilizado nesta etapa será o mesmo carregado através da plataforma.<br>
O tipo da variável retornada depende do arquivo de origem:
- [pandas.DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html) para CSV e compressed CSV: .csv .csv.zip .csv.gz .csv.bz2 .csv.xz
- [Binary IO stream](https://docs.python.org/3/library/io.html#binary-i-o) para outros tipos de arquivo: .jpg .wav .zip .h5 .parquet etc

In [ ]:
import os
import zipfile
root_folder_name = dataset.split("/")[-1].split(".")[0]
root_folder = os.path.join("/tmp/data",root_folder_name)
with zipfile.ZipFile(dataset, 'r') as zip_ref:
    zip_ref.extractall(root_folder)

In [ ]:
image_paths = []
subfolders = []
for path, subdirs, files in os.walk(root_folder):
    for name in files:
        image_paths.append(os.path.join(path, name))
        subfolders.append(name.split("/")[0].split(".")[0])


image_paths, subfolders

In [ ]:
import pandas as pd

df = pd.DataFrame({"input_image":image_paths, "subfolder":subfolders})

X = df.to_numpy()

## Classe MTCNN

In [ ]:
!pip install facenet-pytorch --quiet

In [ ]:
general_parameters = {'root_folder_name':root_folder_name}

model_parameters = {'image_size':image_size,
                    'margin':margin,
                    'min_face_size':min_face_size,
                    'thresholds':[0.6, 0.7, 0.7],
                    'factor':factor,
                    'keep_all':keep_all,
                    'device':device,
                    'seed':seed}

inference_parameters = {'inference_batch_size':inference_batch_size,
                'input_square_transformation_size':input_square_transformation_size}

In [ ]:
!wget https://raw.githubusercontent.com/platiagro/tasks/main/tasks/cv-mtcnn-face-detection/mtcnn.py

In [ ]:
from mtcnn import MTCNN_Model

model = MTCNN_Model(general_parameters, model_parameters, inference_parameters)
df = model.get_result_dataframe(X)

In [ ]:
import matplotlib.pyplot as plt
from platiagro.plotting import plot_data_table

ax = plot_data_table(df)
plt.show()

## Salva resultados da tarefa 

A plataforma guarda o conteúdo de `/tmp/data/` para as tarefas subsequentes.

In [ ]:
from joblib import dump

artifacts = {
    "general_parameters": general_parameters,
    "model_parameters": model_parameters,
    "inference_parameters": inference_parameters

}

dump(artifacts, "/tmp/data/mtcnn.joblib")